# 📦 Chapter 6 Exercise: Bunny Registration Challenge

In this exercise, you will manually disturb the alignment of a 3D model (the Stanford Bunny or any mesh) and then write the code to register it back to its original position.

**Your Goal:**
1.  Load `registration_source.ply`.
2.  Create a copy (`target`) and apply a random transformation (rotation + translation) to it.
3.  Use **ICP (Point-to-Plane)** to register the original `source` to your transformed `target`.
4.  Visualize the result to verify alignment.

In [ ]:
import open3d as o3d
import numpy as np
import copy

# 1. Load Data
source_path = "../DATA/registration_source.ply"
pcd = o3d.io.read_point_cloud(source_path)
pcd.estimate_normals()

# 2. Create Artificial Target
target = copy.deepcopy(pcd)

# Create a transformation matrix (Rotation + Translation)
theta = np.radians(30.0) # 30 degrees rotation
rotation_matrix = np.array([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
])
target.rotate(rotation_matrix, center=(0,0,0))
target.translate((5.0, 2.0, 0.0)) # Shift it away

# Paint them to distinguish
pcd.paint_uniform_color([1, 0.7, 0]) # Yellow
target.paint_uniform_color([0, 0.6, 0.9]) # Blue

print("Showing Misaligned Clouds...")
o3d.visualization.draw_geometries([pcd, target])

## 3. Perform Registration
Since we know the overlap is 100% (it's the same object) and the deviation is not massive, ICP should work well directly. If not, try Global Registration first.

In [ ]:
# TODO: Run ICP Point-to-Plane
threshold = 1.0 # Tune this!

reg_p2l = o3d.pipelines.registration.registration_icp(
    pcd, target, threshold, np.identity(4),
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

print("ICP Fitness:", reg_p2l.fitness)
print("ICP RMSE:", reg_p2l.inlier_rmse)

# TODO: Transform source
pcd.transform(reg_p2l.transformation)

# Visualize
pcd.paint_uniform_color([1, 0, 0]) # Red for aligned
o3d.visualization.draw_geometries([pcd, target], window_name="Aligned Result")

## 4. Challenge
Try increasing the rotation angle or translation distance in Step 2 until ICP fails. At what point does it break?